# 1) Update all data

In [ ]:
# %load ../src/data/get_data.py
import subprocess
import os

import pandas as pd
import numpy as np

from datetime import datetime

import requests
import json
import git

# # Data Understanding
# 
# ## Data Sources
# * RKI, webscraping https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html
# * John Hopkins (GIT) https://github.com/CSSEGISandData/COVID-19.git
# * Rest API to retrieve covid data from NPGEO https://npgeo-corona-npgeo-de.hub.arcgis.com/
# 

# ###  John Hopkins Source

def get_johns_hopkins():
    ''' Get data by a git pull request, the source code has to be pulled first
        Result is stored in the predifined csv structure
    '''
    # git_pull = subprocess.Popen( "/usr/bin/git pull" ,
    #                      cwd = os.path.dirname( 'data/raw/COVID-19/' ),
    #                      shell = True,
    #                      stdout = subprocess.PIPE,
    #                      stderr = subprocess.PIPE )
    # (out, error) = git_pull.communicate()


    # print("Error : " + str(error))
    # print("out : " + str(out))

    g = git.cmd.Git("../data/raw/COVID-19")

    msg = g.pull()
    print(msg)


# ## REST API CALLS


## data request for Germany

def get_current_data_germany():
    ''' Get current data from germany, attention API endpoint not too stable
        Result data frame is stored as pd.DataFrame

    '''
    # 16 states
    #data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

    # 400 regions / Landkreise
    data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_Landkreisdaten/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

    json_object=json.loads(data.content)
    full_list=[]
    
    for pos,each_dict in enumerate (json_object['features'][:]):
        full_list.append(each_dict['attributes'])

    pd_full_list=pd.DataFrame(full_list)
    pd_full_list.to_csv('../data/raw/NPGEO/GER_state_data.csv',sep=';')
    print(' Number of regions rows: '+str(pd_full_list.shape[0]))

if __name__ == "__main__":
    get_johns_hopkins()
    get_current_data_germany()


In [27]:
%run ../src/data/get_data.py


Already up to date.
 Number of regions rows: 412


# 2) Process Pipeline

In [31]:
# %load ../src/data/Process_JohnHopkins_data.py
import pandas as pd
import numpy as np

from datetime import datetime


def store_relational_JH_data_confirmed():
    ''' Transformes the COVID data in a relational data set

    '''

    data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    pd_raw=pd.read_csv(data_path)

    pd_data_base=pd_raw.rename(columns={'Country/Region':'country',
                      'Province/State':'state'})

    pd_data_base['state']=pd_data_base['state'].fillna('no')

    pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)


    pd_relational_model=pd_data_base.set_index(['state','country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )

    pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')

    pd_relational_model.to_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',index=False)
    print(' Number of rows stored: '+str(pd_relational_model.shape[0]))

def store_relational_JH_data_deaths():
    ''' Transformes the COVID data in a relational data set

    '''

    data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
    pd_raw=pd.read_csv(data_path)

    pd_data_base=pd_raw.rename(columns={'Country/Region':'country',
                      'Province/State':'state'})

    pd_data_base['state']=pd_data_base['state'].fillna('no')

    pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)


    pd_relational_model=pd_data_base.set_index(['state','country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'deaths'},
                                                  )

    pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')

    pd_relational_model.to_csv('../data/processed/COVID_relational_deaths.csv',sep=';',index=False)
    print(' Number of rows stored: '+str(pd_relational_model.shape[0]))

def store_relational_JH_data_recovered():
    ''' Transformes the COVID data in a relational data set

    '''

    data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
    pd_raw=pd.read_csv(data_path)

    pd_data_base=pd_raw.rename(columns={'Country/Region':'country',
                      'Province/State':'state'})

    pd_data_base['state']=pd_data_base['state'].fillna('no')

    pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)


    pd_relational_model=pd_data_base.set_index(['state','country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'recovered'},
                                                  )

    pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')

    pd_relational_model.to_csv('../data/processed/COVID_relational_recovered.csv',sep=';',index=False)
    print(' Number of rows stored: '+str(pd_relational_model.shape[0]))

if __name__ == '__main__':

    store_relational_JH_data_confirmed()
    store_relational_JH_data_deaths()
    store_relational_JH_data_recovered()

 Number of rows stored: 62510
 Number of rows stored: 62510
 Number of rows stored: 59455


# 3) Filter and Doubling Rate Calculation

In [39]:
# %load ../src/features/build_features.py
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    #print(y)
    X = np.arange(-1,2).reshape(-1, 1)
    #print(X)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_


    return intercept/slope


def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result


def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

def calc_filtered_data(df_input,filter_on='confirmed'):
    '''  Calculate savgol filter and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'

    df_output=df_input.copy() # we need a copy here otherwise the filter_on column will be overwritten

    pd_filtered_result=df_output[['state','country',filter_on]].groupby(['state','country']).apply(savgol_filter)#.reset_index()

    #print('--+++ after group by apply')
    #print(pd_filtered_result[pd_filtered_result['country']=='Germany'].tail())

    #df_output=pd.merge(df_output,pd_filtered_result[['index',str(filter_on+'_filtered')]],on=['index'],how='left')
    print(df_output.head())
    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')
    #print(df_output[df_output['country']=='Germany'].tail())
    return df_output.copy()

def calc_doubling_rate(df_input,filter_on='confirmed'):
    ''' Calculate approximated doubling rate and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'


    pd_DR_result= df_input.groupby(['state','country']).apply(rolling_reg,filter_on).reset_index()

    pd_DR_result=pd_DR_result.rename(columns={filter_on:filter_on+'_DR',
                             'level_2':'index'})

    #we do the merge on the index of our big table and on the index column after groupby
    df_output=pd.merge(df_input,pd_DR_result[['index',str(filter_on+'_DR')]],left_index=True,right_on=['index'],how='left')
    df_output=df_output.drop(columns=['index'])


    return df_output


def get_daily_list(total_list):
    ''' Calculate Daily change in cases from the cummulative gathered list

        Parameters:
        ----------
        total_list: list
            A python list containing the cummulative cases
        
        Returns:
        ----------
        df_output: list
            the result will be a list containing daily change in values
    '''
    daily_list=[]
    daily_list.append(total_list.pop(0))
    for each in range(len(total_list)):
        if each == 0:
            daily_list.append(total_list[each] - total_list[0])
        else:
            daily_list.append(total_list[each] - total_list[each-1])
    
    return daily_list

def calc_daily_values_all_countries(all_countries):
    df_daily_all= pd.DataFrame()
    for each_country in all_countries:
        daily_list = get_daily_list(list(pd_daily[pd_daily['country']==each_country]['confirmed']))
        df_daily = pd.DataFrame(np.array(daily_list))

        
        df_daily_death = np.array(get_daily_list(list(pd_daily[pd_daily['country']==each_country]['deaths'])))
        df_daily_recov = np.array(get_daily_list(list(pd_daily_recov[pd_daily['country']==each_country]['recovered'])))


        df_daily = df_daily.rename(columns={0:'daily_confirmed'})
        df_daily['daily_deaths'] = df_daily_death
        df_daily['daily_recovered'] = df_daily_recov
        df_daily['date'] = np.array(pd_daily[pd_daily['country']==each_country]['date'])
        df_daily['country'] = np.array(pd_daily[pd_daily['country']==each_country]['country'])
        df_daily_all = pd.concat([df_daily_all,df_daily])

    return df_daily_all




if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))
    pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
    pd_JH_data=pd_JH_data.sort_values('date',ascending=True).copy()

    #test_structure=pd_JH_data[((pd_JH_data['country']=='US')|
    #                  (pd_JH_data['country']=='Germany'))]

    pd_result_larg=calc_filtered_data(pd_JH_data)
    pd_result_larg=calc_doubling_rate(pd_result_larg)
    pd_result_larg=calc_doubling_rate(pd_result_larg,'confirmed_filtered')
    print("Here 1")

    mask=pd_result_larg['confirmed']>100
    pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)
    
    print("Here 2")
    print(pd_result_larg.head())
    
    pd_result_larg = pd_result_larg.reset_index()
    print(pd_result_larg.head())
    
    pd_JH_data_deaths=pd.read_csv('../data/processed/COVID_relational_deaths.csv',sep=';',parse_dates=[0])
    pd_JH_data_deaths=pd_JH_data_deaths.sort_values('date',ascending=True).reset_index(drop=True).copy()
    pd_DR_result_death = pd_JH_data_deaths[['state','country','deaths']].reset_index()
    print(pd_DR_result_death.head())
    pd_result_larg=pd.merge(pd_result_larg,pd_DR_result_death[['index','deaths']],on=['index'],how='left')
    print("here 3")
    pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)

    pd_JH_data_recov=pd.read_csv('../data/processed/COVID_relational_recovered.csv',sep=';',parse_dates=[0])
    pd_JH_data_recov=pd_JH_data_recov.sort_values('date',ascending=True).reset_index(drop=True).copy()
    pd_DR_result_recov = pd_JH_data_recov[['state','date','country','recovered']].reset_index()
    pd_DR_result_recov.to_csv('../data/processed/COVID_final_recov_set.csv',sep=';',index=False)


    pd_daily = pd_result_larg[['state','country','confirmed','date','deaths']].groupby(['country','date']).agg(np.sum).reset_index()
    pd_daily_recov = pd.read_csv('../data/processed/COVID_final_recov_set.csv',sep=';',parse_dates=[0])
    pd_daily_recov = pd_daily_recov[['state','country','recovered','date']].groupby(['country','date']).agg(np.sum).reset_index()

    df_daily_all= calc_daily_values_all_countries(pd_daily['country'].unique())
    df_daily_all = df_daily_all.reset_index()
    df_daily_all.daily_deaths = df_daily_all.daily_deaths.mask(df_daily_all.daily_deaths.lt(0), 0)
    df_daily_all.to_csv('../data/processed/COVID_final_daily_set.csv',sep=';',index=False)
    print("Done")

    

the test slope is: [2.]
Here 1
Here 2
            date    state       country  confirmed  confirmed_filtered  \
0     2020-01-22  Alberta        Canada          0                 0.0   
39715 2020-01-22       no  Korea, South          1                 0.8   
39950 2020-01-22       no        Kosovo          0                 0.0   
40185 2020-01-22       no        Kuwait          0                 0.0   
40420 2020-01-22       no    Kyrgyzstan          0                 0.0   

       confirmed_DR  confirmed_filtered_DR  
0               NaN                    NaN  
39715           NaN                    NaN  
39950           NaN                    NaN  
40185           NaN                    NaN  
40420           NaN                    NaN  
   index       date    state       country  confirmed  confirmed_filtered  \
0      0 2020-01-22  Alberta        Canada          0                 0.0   
1  39715 2020-01-22       no  Korea, South          1                 0.8   
2  39950 2020-01

# 4) Visual Board


In [43]:
# %load ../src/visualization/visualize.py
import pandas as pd
import numpy as np

import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State
import dash_daq as daq
import dash_bootstrap_components as dbc

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import os
print(os.getcwd())
df_input_large=pd.read_csv('../data/processed/COVID_final_set.csv',sep=';')
df_input_recov=pd.read_csv('../data/processed/COVID_final_recov_set.csv',sep=';')
df_input_daily=pd.read_csv('../data/processed/COVID_final_daily_set.csv',sep=';')





fig = go.Figure()

app = dash.Dash(external_stylesheets=[dbc.themes.LUX, 'https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css'])
app.layout = html.Div([

    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data

    Goal of the project is to teach data science by applying a cross industry standard process,
    it covers the full walkthrough of: automated data gathering, data transformations,
    filtering and machine learning to approximating the doubling time, and
    (static) deployment of responsive dashboard.

    ''',style={'text-align':'center','border-style': 'solid'}),


 
    html.Br(),html.Br(),html.Br(),



    dbc.Row([
        ### input + panel
        dbc.Col(md=5, children=[
            dcc.Markdown('''
            ## Select Country for Country level Stats
            '''),


            dcc.Dropdown(
                id='country_drop_down_stats',
                options=[ {'label': each,'value':each} for each in df_input_large['country'].unique()],
                value='India', # which is pre-selected
                multi=False
            ),
            html.Br(),html.Br(),
            html.Div([

                dcc.Tabs(id='tabs-example', value='tab-1', children=[
                    dcc.Tab(   label='Cummulative', value='tab-1', children=[
                        html.Div(children=[
                            dcc.Markdown('''
                            **Scale Modes**
                            '''),
                            dcc.RadioItems(
                                options=[
                                    {'label': 'Uniform', 'value': 'linear'},
                                    {'label': 'Logarithmic', 'value': 'log'},
                                    
                                ],
                                value='linear',
                                id='scale_type',
                                labelStyle={'display': 'inline-block'}
                            ),
                        
                        ]),
                        dcc.Graph( id='multi_graph'),
                        
                    ]),
                    dcc.Tab(   label='Daily', value='tab-2', children=[
                        
                        
                        dcc.Graph( id='multi_graph_daily'),
                        
                    
                    ]),
                ]),




            ]),
            html.Br(),html.Br(),html.Br(),
            #dcc.Graph( id='multi_graph'),

        ]),
        ### plots
        dbc.Col(md=5, children=[
            
            html.Div(id="output-panel", style={'marginLeft': 250, 'marginTop': 250}),
            
        ])
    ]),


    html.Br(),html.Br(),html.Br(),
    html.Div([
    dcc.Markdown('''
    ## Multi-Select Country for Doubling Rate Visualisation
    ''',style={'text-align':'center','border-style': 'solid', 'border-width': 'thin', 'width':'1500px', 'align':'center', 'padding': '10px', 'align-content': 'center'}),

    html.Br(),html.Br(),
    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in df_input_large['country'].unique()],
        value=['US', 'Germany','Italy'], # which are pre-selected
        multi=True,
        style=dict(
                    width='60%',
                    verticalAlign="middle"
                )
    ),

    # dcc.Markdown('''
    #     ## Select Timeline of confirmed COVID-19 cases or the approximated doubling time
    #     '''),
    
    html.Br(),

    dcc.Dropdown(
        id='doubling_time',
        options=[
            
            {'label': 'Timeline Doubling Rate', 'value': 'confirmed_DR'},
            {'label': 'Timeline Doubling Rate Filtered', 'value': 'confirmed_filtered_DR'},
        ],
        value='confirmed_DR',
        multi=False,
        style=dict(
                    width='60%',
                    verticalAlign="middle"
                )
    ),

    dcc.Graph(figure=fig, id='main_window_slope'),


    ])

    



], style={'marginLeft': 50, 'marginTop': 25})


@app.callback(
    Output('multi_graph', 'figure'),
    [Input('country_drop_down_stats', 'value'),
    Input('scale_type', 'value')]
)
def update_cummulative_stacked_plot(country,scale_type):
    


    traces = []
 
    df_plot=df_input_large[df_input_large['country'] == country]

        
    df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date','deaths']].groupby(['country','date']).agg(np.sum).reset_index()
       #print(show_doubling)

    df_plot_recov = df_input_recov[df_input_recov['country'] == country]
    df_plot_recov=df_plot_recov[['state','country','recovered','date']].groupby(['country','date']).agg(np.sum).reset_index()


    fig=make_subplots(rows=4, cols=1,
                subplot_titles=("Total Confirmed", "Total Active ", "Total Recovered", 'Total Deaths'),
                shared_xaxes=False, 
                vertical_spacing=0.1,
    )
    fig.add_trace(go.Scatter(
                        x=df_plot.date,
                        y=df_plot['confirmed'],
                        mode='markers+lines',
                        opacity=0.9
                 ), row=1,col=1
    )
    fig.add_trace(go.Scatter(
                        x=df_plot.date,
                        y=df_plot['confirmed']- (df_plot_recov['recovered']+df_plot['deaths']),
                        mode='markers+lines',
                        opacity=0.9
                 ), row=2,col=1
    )
    fig.add_trace(go.Scatter(
                        x=df_plot_recov.date,
                        y=df_plot_recov['recovered'],
                        mode='markers+lines',
                        opacity=0.9
                        
                 ), row=3,col=1
    )
    fig.add_trace(go.Scatter(
                        x=df_plot.date,
                        y=df_plot['deaths'],
                        mode='markers+lines',
                        opacity=0.9
                 ), row=4,col=1
    )
    
    
    
    
    fig.update_xaxes(type="date",
                    tickangle=-45,
                    nticks=20,
                    tickfont=dict(size=14,color="#7f7f7f"), 
                    row=1, col=1)
    fig.update_xaxes(type="date",
                    tickangle=-45,
                    nticks=20,
                    tickfont=dict(size=14,color="#7f7f7f"), 
                    row=2, col=1)
    fig.update_xaxes(type="date",
                    tickangle=-45,
                    nticks=20,
                    tickfont=dict(size=14,color="#7f7f7f"), 
                    row=3, col=1)
    fig.update_xaxes(type="date",
                    tickangle=-45,
                    nticks=20,
                    tickfont=dict(size=14,color="#7f7f7f"), 
                    row=4, col=1)

    fig.update_yaxes(type=scale_type, row=1, col=1, title='Confirmed infected people')
    fig.update_yaxes(type=scale_type, row=2, col=1, title='Active infected people')
    fig.update_yaxes(type=scale_type, row=3, col=1, title='Recovered people')
    fig.update_yaxes(type=scale_type, row=3, col=1, title='Deaths')

    fig.update_layout(dict (

                width=900,
                height=1500,
                template="plotly_dark"



                
        ))
    return fig



@app.callback(
    Output('multi_graph_daily', 'figure'),
    [Input('country_drop_down_stats', 'value'),
    ]
)
def update_cummulative_stacked_plot(country):

    scale_type = 'linear'
    traces = []
 
    df_plot=df_input_daily[df_input_daily['country'] == country]

        
    #df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date','deaths']].groupby(['country','date']).agg(np.sum).reset_index()
       #print(show_doubling)

    df_plot_recov = df_input_recov[df_input_recov['country'] == country]
    #df_plot_recov=df_plot_recov[['state','country','recovered','date']].groupby(['country','date']).agg(np.sum).reset_index()


    fig=make_subplots(rows=4, cols=1,
                subplot_titles=("Daily Confirmed", "Daily Active ", "Daily Recovered", "Daily Deaths"),
                shared_xaxes=False, 
                vertical_spacing=0.1,
    )
    fig.add_trace(go.Bar(
                        x=df_plot.date,
                        y=df_plot['daily_confirmed'],
                       
                        name=country,
                       
                 ), row=1,col=1
    )
    fig.add_trace(go.Bar(
                        x=df_plot.date,
                        y=df_plot['daily_confirmed'] - (df_plot['daily_recovered']+df_plot['daily_deaths']),
                       
                        name=country,
                        
                 ), row=2,col=1
    )
    fig.add_trace(go.Bar(
                        x=df_plot.date,
                        y=df_plot['daily_recovered'],
                       
                        name=country,
                        
                 ), row=3,col=1
    )
    fig.add_trace(go.Bar(
                        x=df_plot.date,
                        y=df_plot['daily_deaths'],
                        name=country,
                        
                        
                 ), row=4,col=1
    )
    
    

    fig.update_xaxes(type="date", row=1, col=1)
    fig.update_xaxes(type="date", row=2, col=1)
    fig.update_xaxes(type="date", row=3, col=1)
    fig.update_xaxes(type="date", row=4, col=1)

    fig.update_yaxes(type=scale_type, row=1, col=1, title='Confirmed infected people')
    fig.update_yaxes(type=scale_type, row=2, col=1, title='Active infected people')
    fig.update_yaxes(type=scale_type, row=3, col=1, title='Recovered people')
    fig.update_yaxes(type=scale_type, row=3, col=1, title='Deaths')
    
    
    fig.update_layout(dict (

               width=900,
                height=1500,
                template="plotly_dark"

        ))
    return fig



@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('doubling_time', 'value')]
    )
def update_figure(country_list, show_doubling):
    
    if 'doubling_rate' in show_doubling:
        my_yaxis={'type':"log",
               'title':'Approximated doubling rate over 3 days (larger numbers are better #stayathome)'
              }
    else:
        my_yaxis={'type':"log",
                  'title':'Confirmed infected people (source johns hopkins csse, log-scale)'
              }


    traces = []
    for each in country_list:

        df_plot=df_input_large[df_input_large['country']==each]

        if show_doubling=='doubling_rate_filtered':
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.mean).reset_index()
        else:
            df_plot=df_plot[['state','country','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['country','date']).agg(np.sum).reset_index()
       #print(show_doubling)


        traces.append(dict(x=df_plot.date,
                                y=df_plot[show_doubling],
                                mode='markers+lines',
                                opacity=0.9,
                                name=each,
                                
                        )
                )

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        )
    }



# Python function to render output panel
@app.callback(output=Output("output-panel","children"), inputs=[Input('country_drop_down_stats', 'value'),])
def render_output_panel(country):
    total_cases_until_today, active_cases_today,  = 500000, 200000
    peak_color = "white"
    panel = html.Div([
        html.H4(country),
        dbc.Card(body=True, className="text-white bg-primary", children=[
            
            html.H6("Total cases until today:", style={"color":"white"}),
            html.H3("{:,.0f}".format(total_cases_until_today), style={"color":"white"}),
            

            
            html.H6("Active cases today:", style={"color":"white"}),
            html.H3("{:,.0f}".format(active_cases_today), style={"color":"white"}),
            
        
        ])
    ])
    return panel

if __name__ == '__main__':

    app.run_server(debug=True, use_reloader=False)


C:\Users\saisr\Dropbox\Lecture Materials\EDS\eds_covid\notebooks
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
